In [3]:
import torch
from transformers import ResNetConfig
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
import transformers
from torchvision import transforms
from resnet import ResNetForMultiLabel
from resnet import OrganAMNISTDataset, compute_metrics, train_model
import random
import numpy as np
import os 

In [4]:
#SET SEEDS
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

#### Import NPZ by concatenating

In [5]:
import os

def import_data(directory, save_path=None, save=False):

    data = []
    for filename in os.listdir(directory):
        if filename.endswith('.npz'):
            file_path = os.path.join(directory, filename)
            loaded_data = np.load(file_path)
            data.append(loaded_data)

    # concatenate the data from all files
    all_data = {}
    for key in data[0].keys():
        all_data[key] = np.concatenate([d[key] for d in data], axis=0)

    # check the shape of the concatenated data
    for key, value in all_data.items():
        print(f"{key}: {value.shape}")  

    if save:
        if save_path is None:
            save_path = f'datasets/{directory}_concatenated_data.npz'
        np.savez(save_path, **all_data)
        print(f"Data saved to {save_path}")

    return all_data

### Image normalizer 

In [6]:
def normalize_image(image, mean=0.5, std=0.5):
    """
    Normalize an image tensor to have a mean and standard deviation.
    """
    return (image - mean) / std

def normalize_images(images, mean=0.5, std=0.5):
    """
    Normalize a list of images.
    """
    return [normalize_image(image, mean, std) for image in images]

### Dataset Loader

In [7]:
#Modified CustomImageDataset Loader

class ModifiedCustomImageDataset(Dataset):
    def __init__(self, images, labels1,  transform=None):
        self.images = images 
        self.labels1 = labels1

        if transform is None:
            self.transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Lambda(lambda x: x.repeat(3, 1, 1))  # Grayscale to 3-channel
            ])
        else:
            self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx].astype(np.float32)
        label1 = int(self.labels1[idx])


        if self.transform:
            img = self.transform(img)

        return {
            "pixel_values": img,
            "labels": int(label1) if torch.is_tensor(label1) else label1,
        }
        


#### Preprocessing functions for single distortion and multi.

In [8]:
#Modified

def training_preprocess_data(data, float16=False, keylist=['original', 'Uniform_Noise', 'Rotate_90deg', 'Ring_Artifact_v1']):
    """
    Modified to combine the original + distorted images into one set for training C model.
    """
    keys = data.files 
    print(f'Keys found: {keys}')
    
    images = []
    labels = []
    
    for key in keylist: #Iterate through the different distortions
        if key == 'label':
            continue
        images.append(data[key])
        labels.append(data['label'])
    
    normalized_images = []
    for image in images:
        normalized_images.append(normalize_images(image))
        
    labels = np.concatenate(labels, axis=0)
    
    if float16:
        normalized_images = np.concatenate(normalized_images, axis=0).astype(np.float16)
    else:
        normalized_images = np.concatenate(normalized_images, axis=0)
    
    
    print(f"Labels shape: {labels.shape}")
    print(f"Images shape: {normalized_images.shape}")

    dataset = ModifiedCustomImageDataset(images=normalized_images, labels1=labels)

    return dataset


def validation_preprocess_data(data, key='original'):
    """
    Modified from sam's implementation to preprocess 1 set of images. 
    
    key : str (distortion name)
    """
    keys = data.files 
    print(f'\nGenerating {key} validataion set')
    
    images = []
    labels = []

    images.append(data[key])
    labels.append(data['label'])
    
    normalized_images = []
    for image in images:
        normalized_images.append(normalize_images(image))
        
    labels = np.array(labels).squeeze()
    normalized_images = np.array(normalized_images).squeeze()
    
    
    print(f"Labels shape: {labels.shape}")
    print(f"Images shape: {normalized_images.shape}")

    dataset = ModifiedCustomImageDataset(images=normalized_images, labels1=labels)

    return dataset


#### Preprocess validation data into distinct sets

In [9]:
val_set_loaded = np.load('val_concatenated_dataset_full.npz')
key_list = [key for key in val_set_loaded.files if key != 'label']
print(key_list)

val_rotate_set = validation_preprocess_data(val_set_loaded, 'Rotate_90deg')
val_original_set = validation_preprocess_data(val_set_loaded, 'original')
val_noise_set = validation_preprocess_data(val_set_loaded, 'Uniform_Noise')
val_ct_set = validation_preprocess_data(val_set_loaded, 'Ring_Artifact_v1')

['original', 'Uniform_Noise', 'Rotate_90deg', 'Ring_Artifact_v1']

Generating Rotate_90deg validataion set
Labels shape: (6491,)
Images shape: (6491, 224, 224)

Generating original validataion set
Labels shape: (6491,)
Images shape: (6491, 224, 224)

Generating Uniform_Noise validataion set
Labels shape: (6491,)
Images shape: (6491, 224, 224)

Generating Ring_Artifact_v1 validataion set
Labels shape: (6491,)
Images shape: (6491, 224, 224)


### Preprocess training data into one set

In [10]:
keylist=['original', 'Uniform_Noise', 'Rotate_90deg', 'Ring_Artifact_v1']
keylist=['original', 'Uniform_Noise','Ring_Artifact_v1']
train_set_loaded = np.load('training_concatenated_dataset_full.npz')
training_set = training_preprocess_data(train_set_loaded, float16=True, keylist=keylist)

Keys found: ['original', 'label', 'Uniform_Noise', 'Rotate_90deg', 'Ring_Artifact_v1']
Labels shape: (103683, 1)
Images shape: (103683, 224, 224)


## Train Model

In [ ]:
output_path = os.path.join('C11_model', 'results')


config = ResNetConfig()
model = ResNetForMultiLabel(config)

print("Starting training")
trainer = train_model(
    train_dataset=training_set,
    eval_dataset=val_original_set,
    model=model,
    output_dir=output_path,  # Checkpoints will go here
    num_epochs=100,
    batch_size=32
)

print("Saving final model")
trainer.save_model(output_path)

In [12]:

keylist=['original', 'Rotate_90deg']
train_set_loaded = np.load('training_concatenated_dataset_full.npz')
training_set = training_preprocess_data(train_set_loaded, float16=True, keylist=keylist)
output_path = os.path.join('C10_model_r', 'results')


config = ResNetConfig()
model = ResNetForMultiLabel(config)

print("Starting training")
trainer = train_model(
    train_dataset=training_set,
    eval_dataset=val_original_set,
    model=model,
    output_dir=output_path,  # Checkpoints will go here
    num_epochs=100,
    batch_size=32
)

print("Saving final model")
trainer.save_model(output_path)

Keys found: ['original', 'label', 'Uniform_Noise', 'Rotate_90deg', 'Ring_Artifact_v1']
Labels shape: (69122, 1)
Images shape: (69122, 224, 224)
Starting training


  0%|          | 0/216100 [00:00<?, ?it/s]

Could not estimate the number of tokens of the input, floating-point operations will not be computed


{'loss': 1.1112, 'grad_norm': 91.74635314941406, 'learning_rate': 0.099, 'epoch': 1.0}


  0%|          | 0/812 [00:00<?, ?it/s]

...

  0%|          | 0/812 [00:00<?, ?it/s]

{'eval_loss': 0.12896493077278137, 'eval_accuracy': 0.9782776151594516, 'eval_precision': 0.9784043403110506, 'eval_recall': 0.9784245366265657, 'eval_f1': 0.978180281843127, 'eval_runtime': 10.8147, 'eval_samples_per_second': 600.204, 'eval_steps_per_second': 75.083, 'epoch': 24.0}
{'loss': 0.0024, 'grad_norm': 0.012109609320759773, 'learning_rate': 0.07500000000000001, 'epoch': 25.0}


  0%|          | 0/812 [00:00<?, ?it/s]

{'eval_loss': 0.18925584852695465, 'eval_accuracy': 0.9701124634108766, 'eval_precision': 0.9712731234959201, 'eval_recall': 0.9717825268491033, 'eval_f1': 0.9701752664648481, 'eval_runtime': 11.2321, 'eval_samples_per_second': 577.896, 'eval_steps_per_second': 72.293, 'epoch': 25.0}


KeyboardInterrupt: 

In [3]:

if torch.cuda.is_available():
    print("CUDA is available!")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
    device = torch.device("cuda")
else:
    print("CUDA is not available. Using CPU.")
    device = torch.device("cpu")

CUDA is available!
Using GPU: NVIDIA GeForce RTX 3090
